### 문제 텍스트 유사도 분석 워크북

전대일, 2022-06-17

#### 테스트 개요 및 목적
- OOO 기출문제 수집 O만건중 5건을 랜덤 추출하여 주요 개념 리뷰 진행(문서 개수 증가 시 처리 속도가 급격한 증가하여 빠른 개념 테스트가 어렵다.)
- 테스트를 위한 샘플은 동일한 소단원 내에서 선택하여 NLP 텍스트 유사도 분석 기법이 기존 인간이 설정한 범주 하위에서 의미있는 추가 유형화를 만들 수 있는지 테스트한다.
- 기출로 서비스에 활용될 텍스트유사도 데이터 생성을 위한 주요 기술 이슈를 도출한다.


#### 샘플 데이터 로드
- 아래에서 사용하는 questions_math.csv는 엑셀로 제공받은 수학 기출문제 OCR 처리 자료에서 컬럼제목을 1줄로 변경하고 CSV(UTF, 콤마) 저장한 파일이며, 요청시 제공가능함.

In [30]:
import pandas as pd

fulldata = pd.read_csv("questions_math.csv")

temp_df = fulldata.loc[fulldata['소단원'] == '8. 비율에 관한 문제']
del(fulldata)

sample_df = temp_df.sample(5, random_state=1)
del(temp_df)

# 데이터 Preivew
sample_df.head()


,지역명,학교급,학교명,학년,과목,시험년도,학기,시험구분,문항순번,문항Text,대단원,중단원,소단원,유형,하,중하,중,중상,상
44511,경상북도 구미시,중,해마루중학교,2,MATH,2019,1,기말,10,해마루 중학교의 올해 학생 수는 작년에 비하여 남학생 수는 \( 3 \% \) 줄고...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,2,0
6855,인천광역시 부평구,중,산곡남중학교,2,MATH,2018,1,기말,17,작년에 어느 회사의 사원 수는 500 명이었다. 올해는 작년보다 남자 사원이 \( ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,3,0
47340,부산광역시 부산진구,중,동의중학교,2,MATH,2020,1,기말,17,어느 중학교의 올 해 학생 수는 작년보다 남학생 은 \( 10 \% \) 늘고 여학...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,4,0,1
10139,서울특별시 영등포구,중,여의도중학교,2,MATH,2018,1,중간,23,다음 표는 빵과 버터에 들어있는 단백질과 지방의 백분율 \( (\%) \) 이다. ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,4. 합금과 영양소에 관한 문제 (연립일차방정식),0,0,5,0,0
15328,서울특별시 강동구,중,한영중학교,2,MATH,2018,1,중간,19,어느 중학교 작년 입학생 수는 150 명이었다. 올 해 입학한 남학생 수는 작년에 ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,3,0


- 유사도 분석은 '문항Text' 컬럼을 대상으로 진행한다. 해당 값의 형태소를 분리하고, 빈도를 기반으로 수치화(a.k.a. 벡터화)한 뒤, 각 값들이 상호간에 얼마나 유사한지를 수치화하는 작업이다. 
- 대상 컬럼인 수식이 Latex 형태로 들어있어, 이에 대한 의사결정이 필요하나, 이번 테스트에서는 모두 삭제한다. 즉, 영문과 부호는 모두 삭제하고, 한글과 숫자만 남겨둔다.

In [31]:
sample_df['문항Text'] = sample_df['문항Text'].str.replace(r'[^가-힣0-9]', ' ', regex=True) # 전체 데이터 적용시에는 속도 개선을 위한 변경이 필요.
sample_df.head()

,지역명,학교급,학교명,학년,과목,시험년도,학기,시험구분,문항순번,문항Text,대단원,중단원,소단원,유형,하,중하,중,중상,상
44511,경상북도 구미시,중,해마루중학교,2,MATH,2019,1,기말,10,해마루 중학교의 올해 학생 수는 작년에 비하여 남학생 수는 3 줄고...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,2,0
6855,인천광역시 부평구,중,산곡남중학교,2,MATH,2018,1,기말,17,작년에 어느 회사의 사원 수는 500 명이었다 올해는 작년보다 남자 사원이 ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,3,0
47340,부산광역시 부산진구,중,동의중학교,2,MATH,2020,1,기말,17,어느 중학교의 올 해 학생 수는 작년보다 남학생 은 10 늘고 여학...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,4,0,1
10139,서울특별시 영등포구,중,여의도중학교,2,MATH,2018,1,중간,23,다음 표는 빵과 버터에 들어있는 단백질과 지방의 백분율 이다 ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,4. 합금과 영양소에 관한 문제 (연립일차방정식),0,0,5,0,0
15328,서울특별시 강동구,중,한영중학교,2,MATH,2018,1,중간,19,어느 중학교 작년 입학생 수는 150 명이었다 올 해 입학한 남학생 수는 작년에 ...,5. 연립일차방정식의 활용,1. 연립일차방정식의 활용,8. 비율에 관한 문제,"2. 증가, 감소에 관한 문제 (연립일차방정식)",0,0,3,3,0


- 수식이 제거된 문항 Text만 남은 것을 확인할 수 있다.

#### 형태소 분리
- KoNLPy의 2가지 형태소 태그 엔진(OKT, KKma)을 사용해서 비교한다.

In [ ]:
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
okt = Okt()
kkma = Kkma()

- OKT(Open Korean Text)는 기존 KoNLPy에서 Twitter로 불리던 것이 명칭이 변경된 것임
- KKma는 서울대 언어학 연구소에서 제공함

In [39]:
# 테스트용 샘플 문제 텍스트
sample_text = sample_df['문항Text'].iloc[3]
print(sample_text)


다음 표는 빵과 버터에 들어있는 단백질과 지방의 백분율            이다   단백질    92               지방    60              을 섭취하려면 빵과 버터를 각각 몇                  씩 먹으면 되는지 구하여라   구분   단백질   지방 빵      9             2       버터      4              80         1  미지수를 정하시오   2  연립방정식을 세우시오   3  해를 구하시오  


In [45]:
print("OKT 명사추출 결과:\n", okt.nouns(sample_text))
print("KKma 명사추출 결과:\n",kkma.nouns(sample_text))

OKT 명사추출 결과:
 ['다음', '표', '빵', '버터', '단백질', '지방', '백분율', '단백질', '지방', '섭취', '빵', '버터', '각각', '몇', '구', '구분', '단백질', '지방', '빵', '버터', '미지수', '정', '립', '방정식', '해', '구']
KKma 명사추출 결과:
 ['다음', '표', '빵', '버터', '단백질', '지방', '분율', '92', '60', '섭취', '몇', '구', '구분', '분', '9', '2', '4', '80', '1', '미', '미지수', '지수', '연립', '연립방정식', '방정식', '3', '해']


- OKT는 "연립방정식"을 인식하지 못해, "립", "정" 등이 분리되어 추출된다. 또한, 숫자를 모두 무시한다.
- KKma는 "백분율"을 인식하지 못해, "분율"로 표시되고, "미지수"를 "미", "지수", "미지수" 3가지로 모두 반영한다.
- OKT는 명사 반복시 이를 그대로 표현하는 반면, KKma는 반복명사는 한번만 추출한다. 이는 빈도수 기반 유사도 분석에 결정적인 영향을 줄 수 있다.
- 이외에도 몇가지 이슈가 있으며, 아마도 수학과목 문제 텍스트에 특화된 용어사전이 필요한 것으로 보인다.
- 물론, 현재 기획에서는 어차피 단어나 형태소의 의미는 중요하지 않고, 빈도수에 기반한 수치화된 유사도만 필요하기 떄문에 이런 이슈들이 문제가 안될 수도 있다. 최종적으로 제시된 유사한 텍스트 간의 결과로서의 유사성이 중요하지 그 중간 도달과정의 파쇄된 형태소들은 큰 의미가 없을 수 있다. 모든 텍스트에 동일한 조건으로 비교하기 때문에 더더욱. 문제는 이런 판단을 사전에 알기는 어렵고 여러가지 테스트를 해보면서 실증적으로 결정해야할 사안으로 보인다.

#### 맞춤형 용어 등록 방식
- 기존 Okt나 KKma에 용어를 등록하기 위해서는 해당 라이브러리의 jar 파일에 수작업으로 입력한 뒤 jar를 다시 압축해서 등록해야해서 매우 번거롭다.
- Customized KoNLPy를 사용하면 용어를 쉽게 등록할 수 있으나, KoNLPy 보다는 기본 사전에 대한 업데이트가 늦어져 있을 것으로 보여, trade-off 검토가 필요할 수 있다.

In [58]:
import ckonlpy
from ckonlpy.tag import Twitter

CustomTagger = Twitter()


- KoNLPy의 Okt 사용시 추출을 못한 "연립방정식"을 기준으로 테스트를 진행한다.
- 먼저 사전 추가 전에 Customized KoNLPy의 기본 명사추출을 사용하고, 사전에 "연립방정식"을 추가한 뒤 다시 명사 추출 기능을 테스트한다. 

In [59]:
print(CustomTagger.nouns(sample_text))

CustomTagger.add_dictionary('연립방정식', 'Noun')

print(CustomTagger.nouns(sample_text))

['다음', '표', '빵', '버터', '단백질', '지방', '백분율', '단백질', '지방', '섭', '취하', '려면', '빵', '버터', '각각', '몇', '구', '구분', '단백질', '지방', '빵', '버터', '미지수', '정', '시오', '립', '방정식', '세', '우시', '오', '해', '구', '시오']
['다음', '표', '빵', '버터', '단백질', '지방', '백분율', '단백질', '지방', '섭', '취하', '려면', '빵', '버터', '각각', '몇', '구', '구분', '단백질', '지방', '빵', '버터', '미지수', '정', '시오', '연립방정식', '세', '우시', '오', '해', '구', '시오']


- 사전 추가 결과를 바로 비교할 수 있다.
- 하지만, Customized KoNLPy가 일반 KoNLPy 보다 명사 추출 기능이 현저히 떨어진다는 것도 알 수 있다. (동사에 포함된 어근들을 명사로 추출하는 경우가 많다.)


##### 소결

| 형태소        | 장점           | 단점  |
| ------------- |:-------------:| :-----:|
| OKT(KoNLPy)  | 명사 빈도에 따른 반복 | 일부 수학용어 실패, 숫자 무시 |
| KKma(KoNLPy)  | 숫자 인식  |  일부 수학용어 실패, 반복 명사 무시 |
| Custom KoNLPy  | 손쉬운 용어사전 추가 | 명사 추출 성능 |

각각 장단점이 있어 쉽게 결정할 수 없고, 1~2주 정도 여러가지 관점에서 고민이 필요할 것 같다. 수학문제이니 만큼 숫자가 중요할 수 있으나, 어차피 수식을 포함하지 않는다면 의미가 떨어질 수 있다. 용어사전 효율성이 주는 효과와 기본 품사 태깅 성능 간의 tradeoff도 벡터화된 글자들의 의미를 보면 중요하지만, 오직 텍스트 유사도 성능 관점에서도 의미가 있을지 아직은 잘 모르겠다. 하지만, 유사도 외에 키워드나 LDA 토픽 모델링 등 다른 서비스에도 활용되는 벡터화를 염두에 둔다면, Custom 개발을 차분히 지속해야 하고, 이는 KoNLPy에 jar 추가하는 방식일 수도, Custom KoNLPy를 잘 수정하는 것일 수도 있다.

> 더 분석이 필요하지만, 일단은 OKT로 유사도 거리 분석을 진행한다.

#### 단순 Tf-IDF 기반 유사도 분석

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data =[]
for i in range(len(sample_df)):
    text1 = sample_df['문항Text'].iloc[i]
    text1 = okt.nouns(text1)
    text2 =  ' '.join(text1) # 개별 명사가 구분된 리스트를 하나로 스트링으로 조인하여 (마치 하나의 문서처럼 만들어) 비교한다.
    text_data.append(text2)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data) 

print((tfidf_matrix*tfidf_matrix.T).toarray())

[[1.         0.04743577 0.60433973 0.01660117 0.51130282]
 [0.04743577 1.         0.07254548 0.01145361 0.14275481]
 [0.60433973 0.07254548 1.         0.         0.3992814 ]
 [0.01660117 0.01145361 0.         1.         0.        ]
 [0.51130282 0.14275481 0.3992814  0.         1.        ]]


- 가장 유사도가 높은 짝(pair)과 가장 낮은 짝의 원문서(수식 제거 후 문서) 인덱스를 찾아서 비교보자
- 가장 유사: 1번 & 3번 문서
- 가장 다름: 1번 & 4번 문서

In [76]:
print("기준문서:\n", sample_df['문항Text'].iloc[0], "\n")

print("가장 유사한 문서:\n", sample_df['문항Text'].iloc[2], "\n")

print("가장 다른 문서:\n", sample_df['문항Text'].iloc[3])


기준문서:
 해마루 중학교의 올해 학생 수는 작년에 비하여 남학생 수는    3       줄고  여학생 수는   4    줄어서 전체 학생 수는 15명이 적어진 425명이 되었다고 한다  작년의 냠학생 수와 여학생 수를 각각 구하면    1  남학생  180 명  여학생   260 명  2  남학생  220 명  여학생   220 명  3  남학생  200 명  여학생   240 명  4  남학생  240 명  여학생   200 명  5  남학생   260 명  여학생   180 명 

가장 유사한 문서:
 어느 중학교의 올 해 학생 수는 작년보다 남학생 은    10       늘고 여학생은    5       줄어 전체 학생 수가 30 명이 늘었다  올해 전체 학생 수가 930 명 일 때  올 해의 남학생 수는    1  380  2  400  3  500  4  550  5  600 

가장 다른 문서:
 다음 표는 빵과 버터에 들어있는 단백질과 지방의 백분율            이다   단백질    92               지방    60              을 섭취하려면 빵과 버터를 각각 몇                  씩 먹으면 되는지 구하여라   구분   단백질   지방 빵      9             2       버터      4              80         1  미지수를 정하시오   2  연립방정식을 세우시오   3  해를 구하시오  


- 샘플 데이터 5개 내에서는 눈으로 확인해도 OKT품사 태그와 Tf-IDf 기반 단순 유사도가 잘 작동하는 것으로 보인다.

#### 코사인거리 비교
- To Be Done